In [51]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
from sortedcontainers import SortedDict
import seaborn as sns
from  matplotlib.ticker import PercentFormatter
import time

In [52]:
# settings
sns.set_style("darkgrid")
dim = 2

In [53]:
# import node embeddings
df = pd.read_csv(
    f"./data/facebook_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-36_ind-36_ds-0.0_d-{dim}_type-BGMM_k-10.txt",
    sep="\t| ",
    header=None
)
df = df.rename(columns={0: 'node'})
df.set_index(['node'], inplace=True)

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [54]:
# import train
train = pd.read_csv("./data/facebook_train.csv", index_col=0)
# import test
test = pd.read_csv("./data/facebook_test.csv", index_col=['u', 'v'])

In [55]:
# import labels
labels = np.genfromtxt(f"./data/labels_pred_BGMM_d{dim}_k10.txt")
labels

array([1., 1., 1., ..., 6., 6., 6.])

In [56]:
# add labels to embeddings
df['label'] = labels

In [57]:
# node embeddings
display(df.head())
display(df.describe())

# train df
display(train.head())
display(train.describe())

# test df
display(test.head())
display(test.describe())

,1,2,label
node,,,
0,-4.001292,4.017002,1.0
1,-4.303226,4.328349,1.0
2,-3.510165,3.493527,1.0
3,-4.429653,4.469522,1.0
4,-3.582412,3.565915,1.0


,1,2,label
count,3964.000000,3964.000000,3964.000000
mean,-3.137470,-0.250075,2.614026
std,1.930306,3.969619,1.606467
min,-6.907726,-7.603607,1.000000
25%,-4.914396,-3.758388,1.000000
50%,-3.191420,-0.248374,2.000000
75%,-1.127887,3.132293,3.000000
max,-0.400827,6.904348,6.000000


,u,v
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


,u,v
count,86999.000000,86999.000000
mean,1865.069449,2154.507420
std,910.682085,894.143924
min,0.000000,1.000000
25%,1160.000000,1601.000000
50%,1983.000000,2259.000000
75%,2423.000000,2631.000000
max,4031.000000,4038.000000


,,Unnamed: 0
u,v,
0,276,0
9,21,1
21,281,2
25,141,3
26,9,4


,Unnamed: 0
count,1169.000000
mean,584.000000
std,337.605539
min,0.000000
25%,292.000000
50%,584.000000
75%,876.000000
max,1168.000000


In [58]:
def emb_from_row(row):
    """get node embedding np array from df row"""
    return row.drop('label').to_numpy().flatten()

def emb_distance(emb1, emb2):
    """computes the euclidean distance between two node embeddings"""
    return np.linalg.norm(emb1-emb2)

In [59]:
# get nx train graph
G_train = nx.from_pandas_edgelist(train, "u", "v")

In [ ]:
# compute Top-N lists for test dataset
# for each testing row
results = {}
results_com = {}
i = 1
start_time = time.time()
for index, _ in test.iterrows():
    userId = index[0]
    if not userId in df.index: continue
    user_emb = emb_from_row(df.loc[userId])
    user_label = df.loc[userId].label

    # get user's non_friends in user's community
    # users_in_community
    users_in_community = df[df.label == user_label]
    #print("len(users_in_community): ", len(users_in_community))
    # friends
    friends = list(G_train[userId])
    #print("len(friends): ", len(friends))
    # not_friends
    not_friends = df[~df.index.isin(friends+[userId])]
    #print("len(not_friends): ", len(not_friends))
    # not_friends_in_community
    not_friends_in_community = users_in_community[~users_in_community.index.isin(friends+[userId])]
    #print("len(not_friends_in_community): ", len(not_friends_in_community))

    # generate user's top_list from all not_friends
    distances = {}
    top_list = SortedDict()
    for movie_id, node_row in not_friends.iterrows():
        node_emb = emb_from_row(node_row)
        dist = emb_distance(user_emb, node_emb)
        top_list[dist] = movie_id
        distances[movie_id] = dist
    results[userId] = top_list, distances

    # generate user's top_list from all not_friends_in_community
    distances_com = {}
    top_list_com = SortedDict()
    for movie_id, node_row in not_friends_in_community.iterrows():
        node_emb = emb_from_row(node_row)
        dist = emb_distance(user_emb, node_emb)
        top_list_com[dist] = movie_id
        distances_com[movie_id] = dist
    results_com[userId] = top_list_com, distances_com

    if i % 10 == 0:
        elapsed_sec = (time.time() - start_time)
        elapsed_min = elapsed_sec / 60
        print(f"{i}/{len(test)} in {elapsed_min:.2f} min")
        print(f"  => done at {time.ctime(time.time() + ((len(test)-i)/i)*elapsed_sec)}")
    i += 1

In [ ]:
N = 100
hit_rates = []
hit_rates_com = []

for n in range(1, N + 1):

    # counters
    hits = 0
    misses = 0
    hits_com = 0
    misses_com = 0

    for userId in results.keys():
        top_list, _ = results[userId]
        top_list_com, _ = results_com[userId]
        # get Top-N friend suggestions
        top_n = top_list.values()[:n]
        top_n_com = top_list_com.values()[:n]
        # get user's test friend
        test_friend = test.query(f"u == {userId}").index.get_level_values('v')[0]
        # evaluate
        hit = test_friend in top_n
        if hit:
            hits += 1
        else:
            misses += 1
        hit_com = test_friend in top_n_com
        if hit_com:
            hits_com += 1
        else:
            misses_com += 1

        #print(f"Friend suggestion {test_friend} {'found' if hit else 'not found'} in user {userId}'s Top-{N} list: ", top_N)

    hit_rate = hits / (hits+misses)
    hit_rate_com = hits_com / (hits_com+misses_com)
    print(f"{n}: {hits}/{hits+misses} = {hit_rate}")
    print(f"{n}: {hits_com}/{hits_com+misses_com} = {hit_rate_com} (com)")

    hit_rates.append(hit_rate)
    hit_rates_com.append(hit_rate_com)

In [ ]:
# export hit-rates

Ns = np.arange(N) + 1

hit_rate_1_df = pd.DataFrame(
    zip(Ns, hit_rates, ['Algorithm 1' for _ in range(N)]),
    columns=['N', 'hit_rate', 'Algorithm']
)
hit_rate_2_df = pd.DataFrame(
    zip(Ns, hit_rates_com, ['Algorithm 2' for _ in range(N)]),
    columns=['N', 'hit_rate', 'Algorithm']
)
hit_rate_df = pd.concat([hit_rate_1_df, hit_rate_2_df])
hit_rate_df.to_csv(f"./data/hit_rate_df_d-{dim}.csv")


In [ ]:
# hit rates plot

ax = sns.lineplot(x='N', y='hit_rate', hue='Algorithm', data=hit_rate_df)
ax.set(xlabel='N', ylabel='hit rate', ylim=(0, 1))
ax.yaxis.set_major_formatter(PercentFormatter(1))

fig = ax.get_figure()
fig.savefig(f"./plots/hit_rates_d-{dim}.png", format='png', dpi=1000)

In [ ]:
# hit rates plot on log scale

ax = sns.lineplot(x='N', y='hit_rate', hue='Algorithm', data=hit_rate_df)
ax.set(xlabel='N', ylabel='hit rate', ylim=(0, 1), xscale="log")
ax.yaxis.set_major_formatter(PercentFormatter(1))

fig = ax.get_figure()
fig.savefig(f"./plots/hit_rates_d-{dim}_log.png", format='png', dpi=1000)
